# 使用 Azure AI Agent Service 和 Semantic Kernel 進行 RAG

此程式碼片段展示如何使用 `Azure AI Agent Service` 和 `Semantic Kernel` 建立及管理一個 Azure AI 代理，用於檢索增強生成 (RAG)。該代理根據檢索到的上下文處理使用者查詢，並提供準確的回應。


SQLite 版本修復  
如果您遇到以下錯誤：  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

請在您的筆記本開頭取消註解以下程式碼區塊：  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### 匯入套件
以下程式碼匯入所需的套件：


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# 使用 Semantic Kernel 和 Azure AI Agent Service 的檢索增強生成

此範例展示如何使用 **Azure AI Agent Service**，透過結合語言模型與來自上傳文件的特定領域內容，實現 **檢索增強生成 (RAG)**。

### 運作方式

1. **文件上傳**：將包含資訊（Contoso 的旅遊保險政策）的 markdown 文件 (document.md) 上傳至代理服務。

2. **向量存儲建立**：文件被索引到向量存儲中，以啟用其內容的語義搜尋。

3. **代理配置**：使用 `gpt-4o` 模型實例化代理，並設定以下嚴格指令：
   - 僅根據從文件檢索的內容回答問題。
   - 如果問題超出範圍，則拒絕回答。

4. **文件搜尋工具整合**：將 `FileSearchTool` 註冊到代理中，使模型在推理過程中能搜尋並檢索索引文件中的相關片段。

5. **用戶互動**：用戶可以提出問題。如果文件中找到相關資訊，代理會生成基於文件內容的答案。  
   如果沒有，代理會明確回應文件中沒有足夠的資訊。


### 主函數



請確保首先使用 Azure CLI 執行 `az login`，以便在使用 `DefaultAzureCredential` 時提供正確的身份驗證上下文。Azure AI Agent Service 不使用 API 金鑰。


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```

# 重要規則

在開始之前，請務必遵守以下規則：

1. 確保所有的程式碼片段（如 @@INLINE_CODE_1@@ 或 @@CODE_BLOCK_1@@）保持原樣，並且不要進行翻譯。
2. 如果文件中包含任何占位符（例如 @@INLINE_CODE_2@@），請不要更改它們。
3. 請勿翻譯以下標籤：[!NOTE]、[!WARNING]、[!TIP]、[!IMPORTANT]、[!CAUTION]。
4. 保持所有的 Markdown 格式，例如標題、列表、連結等，與原始文件一致。
5. 如果文件中包含任何 URL 或路徑，請保持原樣，不要進行翻譯。

---

## 範例

以下是一個簡單的範例，展示如何正確處理文件中的內容：

### 原始內容

```markdown
[!NOTE]
這是一個重要的提示。

請參考以下程式碼片段：

@@CODE_BLOCK_2@@

更多資訊請參考 [官方文件](https://example.com)。
```

### 翻譯後的內容

```markdown
[!NOTE]
這是一個重要的提示。

請參考以下程式碼片段：

@@CODE_BLOCK_2@@

更多資訊請參考 [官方文件](https://example.com)。
```

---

## 常見問題

### 我應該如何處理程式碼片段？

請確保所有程式碼片段保持原樣，不要進行任何修改或翻譯。

### 如果文件中包含占位符，我應該怎麼辦？

占位符（例如 @@INLINE_CODE_3@@ 或 @@CODE_BLOCK_3@@）應保持不變，請勿進行翻譯或修改。

---

感謝您遵守這些規則！



---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
